In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from IPython.core.debugger import Pdb; pdb = Pdb()


def get_dc_low_order(ts):
    ts_max = len(ts) - 1
    tls_start_offset = max(6, (ts_max - 9))
    
    tls = ts[tls_start_offset : ][::2]
    
    tls_ordered = sorted(tls, key=lambda x: x[1])
    return tls_ordered

def find_up_trend(tl, ts):
    if ts[-1][0] - tl[0] == 3:
        gg = ts[tl[0] + 1]
        dd = ts[tl[0] + 2]
        if ts[-1][1] > gg[1]:
            return True
        else:
            return False
    elif ts[-1][0] - tl[0] == 5:
        zn1_gg = ts[tl[0] + 1]
        zn1_dd = ts[tl[0] + 2]
        zn2_gg = ts[tl[0] + 3]
        zn2_dd = ts[tl[0] + 4]
        if zn1_gg[1] < zn2_gg[1]:
            gg = zn1_gg
        else:
            gg = zn2_gg
        if zn1_dd[1] > zn2_dd[1]:
            dd = zn1_dd
        else:
            dd = zn2_dd
        if ts[-1][1] > gg[1]:
            return True
        else:
            return False
    elif ts[-1][0] - tl[0] >= 7:
        zn1_gg = ts[tl[0] + 1]
        zn1_dd = ts[tl[0] + 2]
        zn2_gg = ts[tl[0] + 3]
        zn2_dd = ts[tl[0] + 4]
        if zn1_gg[1] < zn2_gg[1]:
            gg = zn1_gg
        else:
            gg = zn2_gg
        if zn1_dd[1] > zn2_dd[1]:
            dd = zn1_dd
        else:
            dd = zn2_dd
        if ts[-1][1] > gg[1]:
            zn_ex_hs = ts[tl[0] + 5 : -2][::2]
            # zn_ex_ls = ts[tl[0] + 6 : -1][::2]
            zn_ex_h = min(zn_ex_hs, key=lambda x: x[1])[1]
            # zn_ex_l = max(zn_ex_ls, key=lambda x: x[1])[1]
            if zn_ex_h >= dd[1]:
                return True
            else:
                return False
        else:
            return False
    else:
        raise RuntimeError('Not Handled')

def get_dc_high(ts, tl):
    th_start_offset = 3
    th_end_offset = tl[0] - 3
    
    ths = ts[th_start_offset : th_end_offset + 1][::2]
    th_extreme = max(ths, key=lambda x: x[1])
    return th_extreme

def get_last_centre_interval(sx, sy, ts):
    if len(sx) == 3:
        return [], [], []
    elif len(sx) == 4:
        s0 = sx[2]
        s1 = sy[2]
        e0 = sx[3]
        e1 = sy[3]
        if e0 - s0 == 3:
            return [s0 + 1, e0 - 1], [ts[s0+1]] * 2, [ts[e0-1]] * 2
        elif e0 - s0 >= 5:
            gg = min(ts[s0+1], ts[s0+3])
            dd = max(ts[s0+2], ts[s0+4])
            num = e0 - s0 - 1
            return list(range(s0 + 1, e0)), [gg] * num, [dd] * num
        else:
            raise RuntimeError('Not Handled')
    else:
        raise RuntimeError('Not Handled')

def down_centre_expand_spliter(t_list):
    ts = list(enumerate(t_list))
    tl_ordered = get_dc_low_order(ts)
    
    last_up_trend_found = False
    t_num = 2
    for tl in tl_ordered:
        if tl[0]  == ts[-2][0]:
            continue
        else:
            if find_up_trend(tl, ts):
                last_up_trend_found = True
                t_num = 3
                break
                
    if last_up_trend_found:
        th = get_dc_high(ts, tl)
        return [ts[0][0], th[0], tl[0], ts[-1][0]], [ts[0][1], th[1], tl[1], ts[-1][1]]
    else:
        tl = ts[-2]
        th = get_dc_high(ts, tl)
        return [ts[0][0], th[0], tl[0]], [ts[0][1], th[1], tl[1]]

def plot_trend(y):
    x = list(range(0, len(y)))
    gg = [min(y[1], y[3])] * len(y)
    dd = [max(y[2], y[4])] * len(y)

    plt.figure(figsize=(len(y),4))
    plt.grid()
    plt.plot(x, y)
    plt.plot(x, gg, '--')
    plt.plot(x, dd, '--')
#     sx, sy = down_centre_expand_spliter(y)
#     plt.plot(sx, sy)
#     cs, cgg, cdd = get_last_centre_interval(sx, sy, y)
#     plt.plot(cs, cgg, '--')
#     plt.plot(cs, cdd, '--')
    plt.show()


# y = [0, 100, 60, 130, 70, 120, 40, 90, 50, 140, 85, 105]
# y = [0, 100, 60, 110, 70, 115, 75, 120, 80, 125, 85, 130, 90, 135]
# y = [0, 100, 60, 110, 70, 78, 77, 121, 60, 93, 82, 141, 78, 134]
# y = [0, 110, 70, 100, 60, 100, 78, 90, 53, 109, 56, 141, 99, 106, 89, 99, 93, 141]
# y = [0, 110, 70, 100, 60, 100, 78, 90, 53, 109, 56, 141, 99, 106, 89, 98, 93, 141]
# y = [0, 100, 60, 110, 70, 120, 40, 130, 80, 140, 65, 75]
# y = [0, 100, 60, 110, 70, 120, 40, 130, 55, 140, 65, 75]
# y = [0, 100, 60, 120, 70, 110, 25, 155, 50, 115, 45, 145, 55, 125]

# y = [0, 100, 60, 110, 70, 72, 61, 143, 77, 91, 82, 100, 83, 124, 89, 99]
# y = [0, 110, 70, 100, 60, 80, 60, 72, 56, 146, 77, 105, 87, 128, 68, 119, 57, 116]
# y = [0, 110, 70, 120, 60, 80, 60, 72, 56, 146, 77, 105, 87, 128, 68, 119, 57, 116]

# y = [0, 100, 60, 120, 70, 110, 25, 155, 35, 145, 45, 135]
# y = [0, 100, 60, 120, 70, 110, 25, 155, 35, 145, 45, 146]

# y = [0, 100, 60, 120, 70, 110, 25, 155, 35, 145, 45, 115, 55, 125]
# y = [0, 100, 60, 120, 70, 110, 25, 155, 35, 145, 45, 115, 55, 105]

y = [0, 100, 60, 120, 70, 110, 35, 75, 25, 145, 65, 85]
y1 = [0, 100, 60, 120, 70, 110, 40, 130, 55, 140, 65, 75]
y2 = [0, 100, 60, 120, 70, 110, 40, 130, 80, 140, 65, 75]

for i in range(11, len(y), 2):
    y_slice = y[:(i + 1)]
    plot_trend(y_slice)
    
for i in range(11, len(y1), 2):
    y_slice = y1[:(i + 1)]
    plot_trend(y_slice)
    
for i in range(11, len(y2), 2):
    y_slice = y2[:(i + 1)]
    plot_trend(y_slice)


In [ ]:
# Centre Expand Prototype
%matplotlib inline

import matplotlib.pyplot as plt


y_base = [0, 100, 60, 130, 70, 120, 40, 90, 50, 140, 85, 105, 55, 80]

for i in range(10, len(y_base)):
    y = y_base[:(i + 1)]
    x = list(range(0, len(y)))
    gg = [min(y[1], y[3])] * len(y)
    dd = [max(y[2], y[4])] * len(y)

    plt.figure(figsize=(i,4))
    plt.grid()
    plt.plot(x, y)
    plt.plot(x, gg, '--')
    plt.plot(x, dd, '--')
    if i % 2 == 1:
        sx, sy = down_centre_expand_spliter(y)
        plt.plot(sx, sy)
    plt.show()
    

In [ ]:
# Random Centre Generator
%matplotlib inline

import random
import matplotlib.pyplot as plt

y_max = 150
y_min = 50
num_max = 18

def generate_next(y_list, direction):
    if direction == 1:
        y_list.append(random.randint(max(y_list[2], y_list[4], y_list[-1]) + 1, y_max))
    elif direction == -1:
        y_list.append(random.randint(y_min, min(y_list[1], y_list[3], y_list[-1]) - 1))

# y_base = [0, 100, 60, 110, 70]
y_base = [0, 110, 70, 100, 60]
# y_base = [0, 100, 60, 90, 70]
# y_base = [0, 90, 70, 100, 60]

direction = 1
for i in range(5, num_max):
    generate_next(y_base, direction)
    direction = 0 - direction

print(y_base)
for i in range(11, len(y_base), 2):
    y = y_base[:(i + 1)]
    plot_trend(y)
    

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

# Group 1
# y_base = [0, 100, 60, 110, 70, 99, 66, 121, 91, 141, 57, 111, 69, 111]
# y_base = [0, 100, 60, 110, 70, 105, 58, 102, 74, 137, 87, 142, 55, 128]
# y_base = [0, 100, 60, 110, 70, 115, 75, 120, 80, 125, 85, 130, 90, 135]
# y_base = [0, 100, 60, 110, 70, 120, 80, 130, 90, 140, 50, 75]
# y_base = [0, 100, 60, 110, 70, 114, 52, 75, 54, 77, 65, 100, 66, 87, 70, 116]
# y_base = [0, 100, 60, 110, 70, 72, 61, 143, 77, 91, 82, 100, 83, 124, 89, 99, 89, 105]

# Group 2
# y_base = [0, 110, 70, 100, 60, 142, 51, 93, 78, 109, 60, 116, 50, 106]
# y_base = [0, 110, 70, 100, 60, 88, 70, 128, 82, 125, 72, 80, 63, 119]
# y_base = [0, 110, 70, 100, 60, 74, 66, 86, 57, 143, 50, 95, 70, 91]
# y_base = [0, 110, 70, 100, 60, 77, 73, 122, 96, 116, 82, 124, 69, 129]
# y_base = [0, 110, 70, 100, 60, 147, 53, 120, 77, 103, 56, 76, 74, 92]
# y_base =  [0, 110, 70, 100, 60, 95, 55, 90, 50, 85, 45, 80, 40, 75]
# y_base = [0, 110, 70, 100, 60, 100, 78, 90, 53, 109, 56, 141, 99, 106, 89, 99, 93, 141]

# Group 3
# y_base = [0, 100, 60, 90, 70, 107, 55, 123, 79, 112, 64, 85, 74, 110]
# y_base = [0, 100, 60, 90, 70, 77, 55, 107, 76, 141, 87, 91, 60, 83]
# y_base = [0, 100, 60, 90, 70, 114, 67, 93, 58, 134, 53, 138, 64, 107]
# y_base = [0, 100, 60, 90, 70, 77, 66, 84, 79, 108, 87, 107, 72, 89]
# y_base = [0, 100, 60, 90, 70, 88, 72, 86, 74, 84, 76, 82, 74, 80]

# Group 4
# y_base = [0, 90, 70, 100, 60, 131, 57, 144, 85, 109, 82, 124, 87, 101]
# y_base = [0, 90, 70, 100, 60, 150, 56, 112, 63, 95, 84, 118, 58, 110]
# y_base = [0, 90, 70, 100, 60, 145, 64, 112, 69, 86, 71, 119, 54, 95]
# y_base = [0, 90, 70, 100, 60, 105, 55, 110, 50, 115, 45, 120, 40, 125]

for i in range(11, len(y_base), 2):
    y = y_base[:(i + 1)]
    x = list(range(0, len(y)))
    gg = [min(y[1], y[3])] * len(y)
    dd = [max(y[2], y[4])] * len(y)

    plt.figure(figsize=(i,4))
    plt.title(y)
    plt.grid()
    plt.plot(x, y)
    plt.plot(x, gg, '--')
    plt.plot(x, dd, '--')
    sx, sy = down_centre_expand_spliter(y)
    plt.plot(sx, sy)
    plt.show()
    

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import itertools

h = [3, 4, 5]
l = [0, 1, 2]

hp = tuple(itertools.permutations(h))
lp = tuple(itertools.permutations(l))

print(hp)
print(lp)

y_list = []
for i in range(0, 6):
    for j in range(0, 6):
        y = []
        for k in range(0, 3):
            y.append(lp[i][k])
            y.append(hp[j][k])
        y_list.append(y)
        
for y in y_list:
    x = list(range(0, len(y)))
    gg = [min(y[1], y[3])] * len(y)
    dd = [max(y[2], y[4])] * len(y)

    plt.figure(figsize=(len(y),4))
    plt.grid()
    plt.plot(x, y)
    plt.plot(x, gg, '--')
    plt.plot(x, dd, '--')
    plt.show()
    